In [2]:
#Import libraries
os.chdir("/home/robbe/ionbot/")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass
import OxiAnalysis as OA
import importlib
importlib.reload(OA)
import warnings; warnings.simplefilter('ignore')
import uniprot
import pprint
import sys
import urllib

In [3]:
master = pd.read_csv("/home/robbe/ionbot/mastersets/master_first.csv")
masterpr = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_first_proteins.csv")
mastermod = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_modifications.csv")

In [4]:
#PXD006657 was still included in masterset but won't be used
master = master[master["Accession"] != 6657]
masterpr = masterpr[masterpr["Accession"] != 6657]
mastermod = mastermod[mastermod["Accession"] != 6657]

In [5]:
g = mastermod.groupby(["protein", "uniprot_id", "unexpected_modification", "position"])["#PSMs"].sum().to_frame().reset_index()
g

,protein,uniprot_id,unexpected_modification,position,#PSMs
0,1433B_HUMAN,P31946,[34]Methyl[N-TERM],63,1
1,1433B_HUMAN,P31946,[35]Oxidation[S],47,2
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
4,1433B_HUMAN,P31946,[4]Carbamidomethyl[Y],50,1
...,...,...,...,...,...
97997,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[99998]ICPL_2H(4)[K],258,1
97998,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999992]GIST-Quat_2H(3)[N-TERM],279,3
97999,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999993]GIST-Quat_2H(3)[K],258,13
98000,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999999]Galactosyl[K],258,2


In [6]:
g10 = g[g["#PSMs"] > 10]

In [7]:
g10[g10["unexpected_modification"].isin(OA.modslist)]

,protein,uniprot_id,unexpected_modification,position,#PSMs
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
14,1433E_HUMAN,P62258,[35]Oxidation[Y],49,25
16,1433E_HUMAN,P62258,[35]oxidation[M],221,120
32,1433F_HUMAN,Q04917,[35]Oxidation[Y],49,25
...,...,...,...,...,...
97774,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],147,35
97775,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],283,52
97776,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],479,21
97777,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[Y],155,14


Goal: per modification have a list of proteins, find out accessible surface of the modified residue, collect in a list

In [8]:
moddf = g10[g10["unexpected_modification"] == "[35]Oxidation[Y]"]
moddf[["uniprot_id", "position"]]

,uniprot_id,position
2,P31946,50
14,P62258,49
32,Q04917,49
40,P61981,49
54,P31947,48
...,...,...
97150,sp|TRFE_HUMAN|,536
97151,sp|TRFE_HUMAN|,578
97152,sp|TRFE_HUMAN|,593
97153,sp|TRFE_HUMAN|,669


In [9]:
moddict = dict()
for mod in OA.modslist:
    moddf = g10[g10["unexpected_modification"] == mod]
    #Remove contaminants
    moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
    protein_list = moddf["uniprot_id"].tolist()
    poslist = moddf["position"].tolist()
    pro_poslist = []
    for i in range(len(protein_list)):
        pro_pos = protein_list[i] + "|" + str(poslist[i])
        pro_poslist.append(pro_pos)
    moddict[mod] = pro_poslist

In [10]:
pro_poslist

['P31946|220',
 'P62258|221',
 'Q04917|223',
 'P61981|223',
 'P31947|155',
 'P31947|220',
 'P27348|218',
 'P63104|218',
 'P02763|129',
 'P19652|129',
 'P19652|174',
 'P20848|89',
 'P01009|250',
 'P01009|266',
 'P01009|375',
 'P01009|382',
 'P01009|398',
 'P01009|409',
 'P04217|123',
 'P01023|151',
 'P01023|666',
 'P01023|688',
 'P01023|713',
 'P01023|1314',
 'P01023|1378',
 'P01023|1385',
 'A8K2U0|144',
 'A8K2U0|421',
 'A8K2U0|1116',
 'A8K2U0|1364',
 'A8K2U0|1378',
 'P05067|303',
 'P01011|204',
 'P01011|205',
 'P01011|219',
 'P01011|243',
 'P01011|284',
 'P01011|290',
 'P01011|401',
 'P01011|414',
 'O94911|831',
 'Q9P1F3|1',
 'Q6UWZ7|187',
 'P07108|25',
 'P07108|47',
 'Q4L235|486',
 'P62736|46',
 'P62736|49',
 'P62736|84',
 'P62736|192',
 'P62736|327',
 'Q562R1|45',
 'Q562R1|48',
 'Q562R1|154',
 'Q562R1|191',
 'P60709|44',
 'P60709|47',
 'P60709|82',
 'P60709|153',
 'P60709|190',
 'P60709|305',
 'P60709|325',
 'P68032|46',
 'P68032|49',
 'P68032|84',
 'P68032|192',
 'P68032|327',
 'P63

In [37]:
from pypdb import *
found_pdbs = Query("P01023").search()
found_pdbs[0]

'1BV8'

In [12]:
def download_pdb(pdbcode, datadir, downloadurl="https://files.rcsb.org/download/"):
    """
    Downloads a PDB file from the Internet and saves it in a data directory.
    :param pdbcode: The standard PDB ID e.g. '3ICB' or '3icb'
    :param datadir: The directory where the downloaded file will be saved
    :param downloadurl: The base PDB download URL, cf.
        `https://www.rcsb.org/pages/download/http#structures` for details
    :return: the full path to the downloaded PDB file or None if something went wrong
    """
    pdbfn = pdbcode + ".pdb"
    url = downloadurl + pdbfn
    outfnm = os.path.join(datadir, pdbfn)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return outfnm
    except Exception as err:
        print(str(err), file=sys.stderr)
        return None

In [38]:
from Bio.PDB import PDBParser
import requests as r
from Bio import SeqIO
from io import StringIO
from pypdb import *

d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
pdb_file = download_pdb(found_pdbs[0], '/home/robbe/ionbot/PDB_files')

if pdb_file != None:
    # run parser
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('struct', pdb_file)    

    # iterate each model, chain, and residue
    # printing out the sequence for each chain

    for model in structure:
        PDBseqlist = []
        for chain in model:
            
            for residue in chain:
                if residue.resname in d3to1:
                    PDBseqlist.append(d3to1[residue.resname])
                    PDBseq = "".join(PDBseqlist)


In [39]:
print(found_pdbs[0])

1BV8


In [40]:
print(PDBseq)
print(len(PDBseq))


EEFPFALGVQTLPQTCDEPKAHTSFQISLSVSYTGSRSASNMAIVDVKMVSGFIPLKPTVKMLERSNHVSRTEVSSNHVLIYLDKVSNQTLSLFFTVLQDVPVRDLKPAIVKVYDYYETDEFAIAEYNAPCSKDLGNA
138


In [63]:

# for i in pro_poslist:
cID= 'P01023'                  #i.split("|")[0]

baseUrl="http://www.uniprot.org/uniprot/"
currentUrl=baseUrl+cID+".fasta"
response = r.post(currentUrl)
cData=''.join(response.text)

Seq=StringIO(cData)
pSeq=SeqIO.parse(Seq,'fasta')
pdb_found = Query(cID).search()[0]
for record in pSeq:
    #Uniprotseq = record.seq
    print(record.seq)
    
        


MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEKGCVLLSYLNETVTVSASLESVRGNRSLFTDLEAENDVLHCVAFAVPKSSSNEEVMFLTVQVKGPTQEFKKRTTVMVKNEDSLVFVQTDKSIYKPGQTVKFRVVSMDENFHPLNELIPLVYIQDPKGNRIAQWQSFQLEGGLKQFSFPLSSEPFQGSYKVVVQKKSGGRTEHPFTVEEFVLPKFEVQVTVPKIITILEEEMNVSVCGLYTYGKPVPGHVTVSICRKYSDASDCHGEDSQAFCEKFSGQLNSHGCFYQQVKTKVFQLKRKEYEMKLHTEAQIQEEGTVVELTGRQSSEITRTITKLSFVKVDSHFRQGIPFFGQVRLVDGKGVPIPNKVIFIRGNEANYYSNATTDEHGLVQFSINTTNVMGTSLTVRVNYKDRSPCYGYQWVSEEHEEAHHTAYLVFSPSKSFVHLEPMSHELPCGHTQTVQAHYILNGGTLLGLKKLSFYYLIMAKGGIVRTGTHGLLVKQEDMKGHFSISIPVKSDIAPVARLLIYAVLPTGDVIGDSAKYDVENCLANKVDLSFSPSQSLPASHAHLRVTAAPQSVCALRAVDQSVLLMKPDAELSASSVYNLLPEKDLTGFPGPLNDQDNEDCINRHNVYINGITYTPVSSTNEKDMYSFLEDMGLKAFTNSKIRKPKMCPQLQQYEMHGPEGLRVGFYESDVMGRGHARLVHVEEPHTETVRKYFPETWIWDLVVVNSAGVAEVGVTVPDTITEWKAGAFCLSEDAGLGISSTASLRAFQPFFVELTMPYSVIRGEAFTLKATVLNYLPKCIRVSVQLEASPAFLAVPVEKEQAPHCICANGRQTVSWAVTPKSLGNVNFTVSAEALESQELCGTEVPSVPEHGRKDTVIKPLLVEPEGLEKETTFNSLLCPSGGEVSEELSLKLPPNVVEESARASVSVLGDILGSAMQNTQNLLQMPYGCGEQNMVLFAPNIYVLDYLNETQQLTPEI

In [21]:
pdb_found

'1BV8'

In [42]:
print(Uniprotseq)
print(len(Uniprotseq))

MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEKGCVLLSYLNETVTVSASLESVRGNRSLFTDLEAENDVLHCVAFAVPKSSSNEEVMFLTVQVKGPTQEFKKRTTVMVKNEDSLVFVQTDKSIYKPGQTVKFRVVSMDENFHPLNELIPLVYIQDPKGNRIAQWQSFQLEGGLKQFSFPLSSEPFQGSYKVVVQKKSGGRTEHPFTVEEFVLPKFEVQVTVPKIITILEEEMNVSVCGLYTYGKPVPGHVTVSICRKYSDASDCHGEDSQAFCEKFSGQLNSHGCFYQQVKTKVFQLKRKEYEMKLHTEAQIQEEGTVVELTGRQSSEITRTITKLSFVKVDSHFRQGIPFFGQVRLVDGKGVPIPNKVIFIRGNEANYYSNATTDEHGLVQFSINTTNVMGTSLTVRVNYKDRSPCYGYQWVSEEHEEAHHTAYLVFSPSKSFVHLEPMSHELPCGHTQTVQAHYILNGGTLLGLKKLSFYYLIMAKGGIVRTGTHGLLVKQEDMKGHFSISIPVKSDIAPVARLLIYAVLPTGDVIGDSAKYDVENCLANKVDLSFSPSQSLPASHAHLRVTAAPQSVCALRAVDQSVLLMKPDAELSASSVYNLLPEKDLTGFPGPLNDQDNEDCINRHNVYINGITYTPVSSTNEKDMYSFLEDMGLKAFTNSKIRKPKMCPQLQQYEMHGPEGLRVGFYESDVMGRGHARLVHVEEPHTETVRKYFPETWIWDLVVVNSAGVAEVGVTVPDTITEWKAGAFCLSEDAGLGISSTASLRAFQPFFVELTMPYSVIRGEAFTLKATVLNYLPKCIRVSVQLEASPAFLAVPVEKEQAPHCICANGRQTVSWAVTPKSLGNVNFTVSAEALESQELCGTEVPSVPEHGRKDTVIKPLLVEPEGLEKETTFNSLLCPSGGEVSEELSLKLPPNVVEESARASVSVLGDILGSAMQNTQNLLQMPYGCGEQNMVLFAPNIYVLDYLNETQQLTPEI

In [43]:
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.pairwise2 import format_alignment
alignments = pairwise2.align.globalxx(Uniprotseq, PDBseq)
print(format_alignment(*alignments[0]))

MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEKGCVLLSYLNETVTVSASLESVRGNRSLFTDLEAENDVLHCVAFAVPKSSSNEEVMFLTVQVKGPTQEFKKRTTVMVKNEDSLVFVQTDKSIYKPGQTVKFRVVSMDENFHPLNELIPLVYIQDPKGNRIAQWQSFQLEGGLKQFSFPLSSEPFQGSYKVVVQKKSGGRTEHPFTVEEFVLPKFEVQVTVPKIITILEEEMNVSVCGLYTYGKPVPGHVTVSICRKYSDASDCHGEDSQAFCEKFSGQLNSHGCFYQQVKTKVFQLKRKEYEMKLHTEAQIQEEGTVVELTGRQSSEITRTITKLSFVKVDSHFRQGIPFFGQVRLVDGKGVPIPNKVIFIRGNEANYYSNATTDEHGLVQFSINTTNVMGTSLTVRVNYKDRSPCYGYQWVSEEHEEAHHTAYLVFSPSKSFVHLEPMSHELPCGHTQTVQAHYILNGGTLLGLKKLSFYYLIMAKGGIVRTGTHGLLVKQEDMKGHFSISIPVKSDIAPVARLLIYAVLPTGDVIGDSAKYDVENCLANKVDLSFSPSQSLPASHAHLRVTAAPQSVCALRAVDQSVLLMKPDAELSASSVYNLLPEKDLTGFPGPLNDQDNEDCINRHNVYINGITYTPVSSTNEKDMYSFLEDMGLKAFTNSKIRKPKMCPQLQQYEMHGPEGLRVGFYESDVMGRGHARLVHVEEPHTETVRKYFPETWIWDLVVVNSAGVAEVGVTVPDTITEWKAGAFCLSEDAGLGISSTASLRAFQPFFVELTMPYSVIRGEAFTLKATVLNYLPKCIRVSVQLEASPAFLAVPVEKEQAPHCICANGRQTVSWAVTPKSLGNVNFTVSAEALESQELCGTEVPSVPEHGRKDTVIKPLLVEPEGLEKETTFNSLLCPSGGEVSEELSLKLPPNVVEESARASVSVLGDILGSAMQNTQNLLQMPYGCGEQNMVLFAPNIYVLDYLNETQQLTPEI

In [ ]:
string = "-QIPLCANL-VPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKP-ETTKEQLGEFYEALDCLRIPRSDVMYTDWKKDKCEPLEKQHEKEIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCL-RIPRSDVMYTDWKKDKCEPLEKQH---------E-"
string.replace("-","")[320:330]

'CLRIPRSDVM'

In [44]:
from Bio import pairwise2

def seqmatch(alignment, pos): #TODO do sanity check met if nieuw gemapte AA == doel AA of check nog even hoe beter op te lossen
    pos = pos - 1
    matches = []

    for i, (a, b) in enumerate(zip(alignment[0], alignment[1])):
        try:
            if alignment.seqA[pos] == alignment.seqB[pos]:
                PDBseq2B = alignment.seqB[0:pos+1]
                countB = PDBseq2B.count("-")
                PDBseq2A = alignment.seqA[0:pos+1]
                countA = PDBseq2A.count("-")
                PDBseq2 = alignment.seqB.replace('-', "")
                if countA > countB:
                    need_added = countA - countB
                    PDBpos = pos + need_added
                elif countA < countB:
                    need_substracted = countB - countA
                    PDBpos = pos - need_substracted
                else:
                    PDBpos = pos
                #PDBpos = pos - count #get index of the AA out of this
                return PDBpos
            else:
                return None
        except IndexError:
                return None
        #     return True
        # else:
        #     return False

m = seqmatch(alignments[0], 1385)
print(m)

48


In [47]:
print(Uniprotseq[1384])

M


In [45]:
print(PDBseq[48])

M


In [27]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
model = structure[0]
dssp = DSSP(model, pdb_file)

In [28]:
a_key = list(dssp)
for i in a_key:
    if i[1] == "X":
        a_key.remove(i)

In [29]:
test = []
for i in a_key:
    test.append(i[1])

In [30]:
test[218]

IndexError: list index out of range

In [31]:
DSSPseqnoX = "".join(test)

In [32]:
a_key[218][1]

IndexError: list index out of range

In [ ]:
moddict = dict()
for mod in OA.modslist:
    moddf = g10[g10["unexpected_modification"] == mod]
    pattern = re.compile(r"\[(.*?)\]")
    AA = re.findall(pattern= pattern, string= mod)[1]
    print(mod, AA)
    #Remove contaminants
    moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
    protein_list = moddf["uniprot_id"].tolist()
    poslist = moddf["position"].tolist()
    pro_poslist = []
    for i in range(len(protein_list)):
        pro_pos = protein_list[i] + "|" + str(poslist[i])
        pro_poslist.append(pro_pos)
    moddict[mod] = pro_poslist

In [69]:
from pypdb import *
from Bio.PDB import PDBParser
import requests as r
from Bio import SeqIO
from io import StringIO
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

mastermod = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_modifications.csv")
mastermod = mastermod[mastermod["Accession"] != 6657]
g = mastermod.groupby(["protein", "uniprot_id", "unexpected_modification", "position"])["#PSMs"].sum().to_frame().reset_index()
g10 = g[g["#PSMs"] > 10]
g10[g10["unexpected_modification"].isin(OA.modslist)]


os.chdir("/home/robbe/ionbot/RSA_files")

# for each mod, get the peptidoforms that had at least 10 PSMs over all datasets
for mod in OA.modslist:
    moddf = g10[g10["unexpected_modification"] == mod]
    #get modified AA for current modification
    pattern = re.compile(r"\[(.*?)\]")
    AA = re.findall(pattern= pattern, string= mod)[1]
    print(mod, AA)
    #Remove peptidoforms from contaminant proteins
    moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
    #get list of uniprot_ids that carry current mod
    protein_list = moddf["uniprot_id"].tolist()
    #get list of modification position
    poslist = moddf["position"].tolist()
    #combine together, e.g. P105486|65 means that protein P105486 carried the modification at site 65
    pro_poslist = []
    for i in range(len(protein_list)):
        pro_pos = protein_list[i] + "|" + str(poslist[i])
        pro_poslist.append(pro_pos)
        print(len(pro_poslist))

       
    check_list = []
    for i in pro_poslist:
        #Get protein and position
        protein = i.split("|")[0]
        pos = int(i.split("|")[1])
        print(protein, pos)
        #Search pdb accession id's for the protein
        found_pdbs = Query(protein).search()
        
        #Get Uniprot sequence for protein
        baseUrl="http://www.uniprot.org/uniprot/"
        currentUrl=baseUrl+protein+".fasta"
        response = r.post(currentUrl)
        cData=''.join(response.text)

        Seq=StringIO(cData)
        pSeq=SeqIO.parse(Seq,'fasta')
        
        for record in pSeq:
            Uniprotseq = record.seq
        #first search result in PDB for Uniprot Accession gives PDB ID
        if found_pdbs != None:
            PDB_ID = found_pdbs[0]
            #dictionary to convert PDB file sequence to one-letter code
            d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
            'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
            'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
            'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
            #Download PDB file
            pdb_file = download_pdb(PDB_ID, '/home/robbe/ionbot/PDB_files')

            #For large structures, PDB file might not be available, and for those, DSSP can't be used
            if pdb_file != None:
                # run parser
                parser = PDBParser(QUIET=True)
                structure = parser.get_structure('struct', pdb_file)    

                # iterate each model, chain, and residue
                # printing out the sequence for each chain
                #Together forms PDBseq
                for model in structure:
                    PDBseqlist = []
                    for chain in model:
                        
                        for residue in chain:
                            if residue.resname in d3to1:
                                PDBseqlist.append(d3to1[residue.resname])
                                PDBseq = "".join(PDBseqlist)
                
                #PDB sequence and Uniprot sequence often don't fully correspond. Because ionbot uses Uniprot seq to localize modification site, but DSSP algorithm uses PDB seq, 
                #The modification site for Uniprot seq first needs to mapped onto the site for PDB seq, to do this, we align the sequences
                alignments = pairwise2.align.globalxx(Uniprotseq, PDBseq)

                #seqmatch gives the index of the modification site on the PDB sequence
                m = seqmatch(alignments[0], pos)
                model = structure[0]
                
                try:
                    dssp = DSSP(model, pdb_file)
                
                    a_key = list(dssp)
                    #DSSP sometimes has unknown residues include in the sequence (denoted as X), if these are removed, you are left with the same as the PDBseq
                    for i in a_key:
                        if i[1] == "X":
                            a_key.remove(i)
                    #if matched amino acid matches the modified amino acid (i.e. alignment was succesful), append RSA to list
                    if m != None:
                        if a_key[m][1] == AA:
                            RSA = a_key[m][3]
                            check_list.append(RSA)
                except Exception:
                    continue
    #Print all RSA's to file                    
    with open("{}_RSA.txt".format(mod),'w') as f:
        for i in check_list:
            f.write(str(i))
            f.write("\n")
            



[35]Oxidation[T] T
P02763 127
P02763 176
P19652 176
P01009 292
P01009 318
P01009 320
P04217 102
P04217 147
P04217 231
P02768 71
P02768 92
P02768 100
P02768 149
P02768 190
P02768 260
P02768 444
P02768 446
P02768 502
P02768 530
P02768 532
P02760 52
P02760 64
P02760 67
P02760 75
P02760 82
P02760 94
P02760 145
P02760 181
P0DUB6 329
P19961 329
P04746 329
P04083 223
P08519 90
P05090 54
P05090 55
P05090 168
P05090 178
P05090 180
P08571 131
P08571 197
P00450 76
P00450 429
P01024 144
P12109 208
P39059 86
P01040 83
Q15828 90
P08174 59
P08174 62
P59665 82
P59666 82
P24855 84
P01133 615
Q9UNN8 105
Q9UNN8 111
Q01469 68
P02765 197
P02765 319
P02751 1204
P02751 1373
Q92820 230
P69905 42
P68871 39
P02042 39
P02100 39
P69891 39
P69892 39
P02790 174
P02790 336
P02790 394
P00739 164
P00738 222
Q16270 230
P05155 394
P05155 397
P0DOX2 263
P0DOX2 380
P0DOX5 262
P0DOX5 352
P0DOX5 368
P01876 205
P01876 278
P01877 148
P01877 265
P01857 143
P01857 233
P01857 249
P01859 22
P01859 229
P01859 245
P01860 280
P01860

KeyboardInterrupt: 

In [58]:
check_list

['P62258|221==>218:M',
 'P31947|155==>159:M',
 'P31947|220==>224:M',
 'P27348|218==>217:M',
 'P63104|218==>214:M',
 'P02763|129==>109:M',
 'P01009|250==>206:M',
 'P01009|266==>222:M',
 'P01009|375==>331:M',
 'P01009|398==>354:M',
 'P01009|409==>365:M']

In [ ]:
pro_poslist

['P31946|220',
 'P62258|221',
 'Q04917|223',
 'P61981|223',
 'P31947|155',
 'P31947|220',
 'P27348|218',
 'P63104|218',
 'P02763|129',
 'P19652|129',
 'P19652|174',
 'P20848|89',
 'P01009|250',
 'P01009|266',
 'P01009|375',
 'P01009|382',
 'P01009|398',
 'P01009|409',
 'P04217|123',
 'P01023|151',
 'P01023|666',
 'P01023|688',
 'P01023|713',
 'P01023|1314',
 'P01023|1378',
 'P01023|1385',
 'A8K2U0|144',
 'A8K2U0|421',
 'A8K2U0|1116',
 'A8K2U0|1364',
 'A8K2U0|1378',
 'P05067|303',
 'P01011|204',
 'P01011|205',
 'P01011|219',
 'P01011|243',
 'P01011|284',
 'P01011|290',
 'P01011|401',
 'P01011|414',
 'O94911|831',
 'Q9P1F3|1',
 'Q6UWZ7|187',
 'P07108|25',
 'P07108|47',
 'Q4L235|486',
 'P62736|46',
 'P62736|49',
 'P62736|84',
 'P62736|192',
 'P62736|327',
 'Q562R1|45',
 'Q562R1|48',
 'Q562R1|154',
 'Q562R1|191',
 'P60709|44',
 'P60709|47',
 'P60709|82',
 'P60709|153',
 'P60709|190',
 'P60709|305',
 'P60709|325',
 'P68032|46',
 'P68032|49',
 'P68032|84',
 'P68032|192',
 'P68032|327',
 'P63